In [21]:
import pandas as pd
from pathlib import Path

In [22]:
DATA_PATH = Path("../data/silver/combined_data.csv")

In [23]:
df = pd.read_csv(DATA_PATH)

In [24]:
# Ensure year is int
df['year'] = df['year'].astype(int)

# Get feature columns (excluding RA and year)
feature_cols = [col for col in df.columns if col not in ['RA', 'year']]

# Find the last year for each RA
last_year_per_ra = df.groupby('RA')['year'].max()

In [25]:
# Create the gold dataframe
gold_data = []

for ra in df['RA'].unique():
    ra_df = df[df['RA'] == ra].sort_values('year')
    last_year = ra_df['year'].max()
    
    # Get the row with the last year
    last_year_data = ra_df[ra_df['year'] == last_year].iloc[0]
    
    row_dict = {'RA': ra}
    
    # Add current year features
    for col in feature_cols:
        row_dict[col] = last_year_data[col]
    
    # Add lagged features
    years_available = sorted(ra_df['year'].unique())
    for i, year in enumerate(years_available):
        year_data = ra_df[ra_df['year'] == year].iloc[0]
        lag = last_year - year
        if lag > 0:
            for col in feature_cols:
                row_dict[f"{col}__minus_{lag}"] = year_data[col]
    
    gold_data.append(row_dict)

df_gold = pd.DataFrame(gold_data)

In [38]:
# Reorder columns: RA first, then all current features, then all minus_1, then all minus_2, etc.
ordered_cols = ['RA']

# Add current year features
for col in feature_cols:
    ordered_cols.append(col)

# Add lagged features by lag level
lag = 1
while True:
    lag_cols = [f"{col}__minus_{lag}" for col in feature_cols if f"{col}__minus_{lag}" in df_gold.columns]
    if not lag_cols:
        break
    ordered_cols.extend(lag_cols)
    lag += 1

df_gold = df_gold[ordered_cols]

In [42]:
# Create a column with the number of year (years of data) for each RA
def count_years(row):
    # Check if minus_2 columns have data (3 years)
    minus_2_cols = [col for col in row.index if '__minus_2' in col]
    if minus_2_cols and pd.notna(row[minus_2_cols[0]]):
        return 3
    
    # Check if minus_1 columns have data (2 years)
    minus_1_cols = [col for col in row.index if '__minus_1' in col]
    if minus_1_cols and pd.notna(row[minus_1_cols[0]]):
        return 2
    
    # Only current year (1 year)
    return 1


In [43]:
df_gold['years'] = df_gold.apply(count_years, axis=1)

print(f"\nYears distribution:")
print(df_gold['years'].value_counts().sort_index())


Years distribution:
years
1    752
2    380
3    444
Name: count, dtype: int64


In [44]:
df_gold.to_csv("../data/gold/gold_data.csv", index=False)